In [20]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate();
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark


In [19]:
#sc.stop()

In [3]:
import os
import tweepy as tw

In [4]:
consumer_key = 'H8J4lTPwHny8qt41OOxLFwNHH'
consumer_secret = 'KNTd6Kida34CBydCJYGm5leKBChBN86sTlMpkFZ6bkfkWAjaYb'
access_token = '3105826550-F4YMOB6ZljoceFpW2WTJctwxAFH757cEy6QwNXD'
access_secret = 'JJaTqeVLPqma5OtYsDwjTldEP8S8qqiRz4VcTmXGf1dwZ'

In [5]:
auth = tw.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)
api= tw.API(auth,wait_on_rate_limit =True)

In [6]:
tweets = tw.Cursor(api.search, q="#Tokyo2020", since = "2021-07-23", until = "2021-08-08",lang="en").items(1)
[[tweet] for tweet in tweets]

[[Status(_api=<tweepy.api.API object at 0x00000264C67D5E50>, _json={'created_at': 'Sat Aug 07 23:59:59 +0000 2021', 'id': 1424158412288319489, 'id_str': '1424158412288319489', 'text': 'RT @BBCSport: Just when you thought speed climbing was frightening enough, you now need to impress a Transformer too at #Tokyo2020! 👀🤖 http…', 'truncated': False, 'entities': {'hashtags': [{'text': 'Tokyo2020', 'indices': [120, 130]}], 'symbols': [], 'user_mentions': [{'screen_name': 'BBCSport', 'name': 'BBC Sport', 'id': 265902729, 'id_str': '265902729', 'indices': [3, 12]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1073899589302800384, 'id_str': '1073899589302800384', 'name': 'プロ雨蛙', 'screen_name

In [7]:
tweets = tw.Cursor(api.search, q="#Tokyo2020", since = "2021-07-23", until = "2021-08-08",lang="en").items(500)
users_details = [[tweet.text,tweet.user.screen_name,tweet.user.name] for tweet in tweets]
users_details

[['RT @BBCSport: Just when you thought speed climbing was frightening enough, you now need to impress a Transformer too at #Tokyo2020! 👀🤖 http…',
  'Windows_frog10',
  'プロ雨蛙'],
 ['RT @TimeOutLondon: The Tokyo Olympics has a new⭐\n\nThis man arrives outside the Olympic Village at 7am each day to support Olympians 😭\n\n#Tok…',
  'DanielleConk',
  'Danielle: Vaccinated, Masked, Afraid of Dying'],
 ['RT @TeamIreland: #Athletics \n\nThrough 35km two Irishmen still battling through the field, picking up places as the race goes on:\n\n🟢 Seaward…',
  'AthleticsNI',
  'Athletics NI'],
 ["It's Eliud Kipchoge's world... #Tokyo2020 https://t.co/wRDOyJ6uPP",
  'EarlAuthours',
  'Makhobalo Lesaoana'],
 ["RT @Olympics: Eliud Kipchoge is a marathon legend and an Olympic champion.\n\nHere’s a look back at his gold at Rio 2016 🥇 Can he win today's…",
  'LadyFear1',
  'Lady Fear'],
 ['RT @celtics: Gold Medal Jay 🥇🍾\n\n@usabasketball x #Tokyo2020 https://t.co/GTPqhLidzh',
  'jmoneybthename_',
  'jmoney🇭🇹🌐

In [8]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
schema = StructType([ \
    StructField("Tweet",StringType(),True), \
    StructField("Username",StringType(),True), \
    StructField("Name", StringType(), True), \
  ])

In [12]:
df = spark.createDataFrame(data=users_details,schema=schema)
df.show(500)

+--------------------+---------------+-------------------------------+
|               Tweet|       Username|                           Name|
+--------------------+---------------+-------------------------------+
|RT @BBCSport: Jus...| Windows_frog10|                       プロ雨蛙|
|RT @TimeOutLondon...|   DanielleConk|           Danielle: Vaccina...|
|RT @TeamIreland: ...|    AthleticsNI|                   Athletics NI|
|It's Eliud Kipcho...|   EarlAuthours|             Makhobalo Lesaoana|
|RT @Olympics: Eli...|      LadyFear1|                      Lady Fear|
|RT @celtics: Gold...|jmoneybthename_|                   jmoney🇭🇹🌐|
|I choose to see a...|         dnkoni|                        Morai_N|
|RT @Olympics: HIS...|   ginaisgolden|           gina♡free palesti...|
|RT @Olympics: A s...|RentiyaMahendr1|           Rentiya Mahendrak...|
|RT @RealBaba_e_Qu...|         IvsCol|                       ColSodhi|
|RT @1bbradfo: It'...|       Mbatiah_|                              E|
|RT @airnewsa